In [1]:
#учит LLM в сборе. То есть приделываем в конец feature head и так и учим.
#особенность в том, что мы накладываем адаптер на последовательность, а не один эмбеддинг
#в даннм случае адаптер - это конволюционка, перемалывающая несколько последних эмбеддингов

In [2]:
install = False

if install:
    !conda install -c anaconda git -y

    !pip install deepspeed==0.14.4

    !pip install --upgrade pip
    !pip install --upgrade transformers==4.43.3 tyro==0.9.8 triton==2.3.1 trl==0.8.6

    !pip install -v "xformers==0.0.29.post1"
    !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
    !pip install --no-deps -v "peft==0.13.2"
    !pip install --no-deps packaging ninja einops flash-attn bitsandbytes
    !pip install accelerate==0.34.2
    !curl -fsSL https://ollama.com/install.sh | sh
    !pip uninstall peft -y
    !pip install -v "peft==0.13.2"

In [3]:
import sys
import time
import json
import os
from collections import OrderedDict
from pathlib import Path
from collections import deque
import os, pickle, random
import psutil
import warnings
import codecs
import numpy as np
import pandas as pd
import requests
from typing import Dict, List, Optional
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModel
from transformers import StoppingCriteria, StoppingCriteriaList
from torch import cuda, LongTensor, FloatTensor
from peft import PeftModel, PeftConfig, PeftModelForCausalLM

import slider_utils
import generate_utils
import ensembles

device = torch.device("cuda")

In [4]:
torch.tensor(1).to(device)

tensor(1, device='cuda:0')

In [5]:
import time
#time.sleep(5*60)

In [6]:
start_train = True#Запускаем ли мы сейчас обучение с нуля, или с какого-то чекпоинта


bits_per_number = 4#Насколько сильно квантуем модель
use_gguf = False#Можно использовать gguf
gguf_folder_name = "gguf_31"#из этой папки

cardinality = 128256#размер словаря токенов
learnable_linear_model = False#учим ли мы линейную субмодель
learnable_all = False#учим ли мы вообще весь трансформер (это очень тяжело по GPU-памяти, я не осилил)

cfg_switch = 1
if cfg_switch == 1:
    #ЛЕГКОВЕСНЫЙ ТРАНСФОРМЕР
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"
    architecture = 'A'
    opt_type = 'adam'
    mode = 'pretrain'#'finetune', 'pretrain'
    #1.403
else:
    #ТЯЖЁЛЫЙ ТРАНСФОРМЕР
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
    architecture = 'I'
    opt_type = 'adam'
    mode = 'pretrain'


tokenizer_name = model_name
padding_token = 128009
forbidden_tokens_list = [padding_token]
seed = int(np.random.rand() * 1000000)#random seed for data sampling

In [7]:
if mode == 'pretrain':
    #претрейн
    # max_seq_len = 6900
    # max_seq_len4inference = 6850
    # max_seq_len = 4500
    # max_seq_len4inference = 4450
    max_seq_len = 3800
    max_seq_len4inference = 3750
    # max_seq_len = 3600
    # max_seq_len4inference = 3550
else:
    #дообучение
    # max_seq_len = 1950
    # max_seq_len4inference = 1900
    max_seq_len = 2050
    max_seq_len4inference = 2000

In [8]:
if model_name == "unsloth/Llama-3.2-1B-Instruct-bnb-4bit":
    embedding_size = 2048#это просто свойство исходной сетки
    #сколько субмоделей выбрасывается из forward на обучении. Чем больше, тем лучше защита от оверфита. Если одна-две субмодели, net_dropout лучше занулять
    net_dropout_rate = 0.0#если десятки субмоделей и хотим огромную защиту от оверфита, то это число надо проставлять в 0.9-0.95
    #дропаут внутри субмоделей
    individ_dropout_rate = 0.05
    conservativity = 0.13#насколько сильно подавляем отклонение от старой стратегии
    accum_batch = 100#60#для дачала сделайте 2
    lr = 2e-5
    
    #layer_configs = [4096, 4096, 2048, 2048, 2048, 2048, 2048, 2048, 1024, 1024, 1024, 1024, 1024]#версия A
    layer_configs = [2048] * 6 + [1024] * 6#версия A
    use_memnets = True
    memnet_params={'num_heads':8, 'query_size':64, 'num_key_values':350, 'value_size':256}
    composition_size = 1#число субмоделей класса ResNet. Чем больше их, тем больше или защита от оверфита (если большой net_dropout), или ёмкость
    sample_features = 1.#какая доля фичей приходит в субмодель. По дефолту ставится 0.4-0.6, но это если субмоделей как минимум 15. Сейчас их 1.
    path2model = "ern_model_A_composed.pth"
    max_tokens_in_loss = 150#сколько пар эмбеддинг-токен проходит через tail adapter за один forward
    batch_size = 1#2
    
    
    window_sizes = [12] * 5
    squeeze_dim = embedding_size//2#выходной размер слайдера
elif model_name == "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit":
    embedding_size = 4096#это просто свойство исходной сетки

    if architecture == 'D':
        #сколько субмоделей выбрасывается из forward на обучении. Чем больше, тем лучше защита от оверфита. Если одна-две субмодели, net_dropout лучше занулять
        net_dropout_rate = 0.0#если десятки субмоделей и хотим огромную защиту от оверфита, то это число надо проставлять в 0.9-0.95
        #дропаут внутри субмоделей
        individ_dropout_rate = 0.2
        conservativity = 0.12#насколько сильно подавляем отклонение от старой стратегии
        accum_batch = 4200#60#для дачала сделайте 2
        lr = 3e-6
        
        layer_configs = [1024, 512, 512, 512, 256, 256, 256, 256, 128, 128, 128, 128]#версия C
        layer_configs = [2048, 1024, 1024, 512, 512, 512, 256, 256, 256, 256, 256, 256, 256, 256, 256]#версия D
        composition_size = 1#число субмоделей класса ResNet. Чем больше их, тем больше или защита от оверфита (если большой net_dropout), или ёмкость
        sample_features = 1.#какая доля фичей приходит в субмодель. По дефолту ставится 0.4-0.6, но это если субмоделей как минимум 15. Сейчас их 1.
        path2model = "ern_model_D_composed_slider.pth"
        max_tokens_in_loss = 5500#сколько пар эмбеддинг-токен проходит через tail adapter за один forward
    elif architecture == 'E':
        #сколько субмоделей выбрасывается из forward на обучении. Чем больше, тем лучше защита от оверфита. Если одна-две субмодели, net_dropout лучше занулять
        net_dropout_rate = 0.0#если десятки субмоделей и хотим огромную защиту от оверфита, то это число надо проставлять в 0.9-0.95
        #дропаут внутри субмоделей
        individ_dropout_rate = 0.05
        conservativity = 0.15#насколько сильно подавляем отклонение от старой стратегии
        accum_batch = 2300#60#для дачала сделайте 2
        lr = 1e-3
        
        layer_configs = [2048, 2048, 1024, 1024, 1024, 512, 512, 512, 512, 512, 512, 512, 256, 256, 256, 256]#версия E
        composition_size = 1#число субмоделей класса ResNet. Чем больше их, тем больше или защита от оверфита (если большой net_dropout), или ёмкость
        sample_features = 1.#какая доля фичей приходит в субмодель. По дефолту ставится 0.4-0.6, но это если субмоделей как минимум 15. Сейчас их 1.
        path2model = "ern_model_E_composed_slider.pth"
        max_tokens_in_loss = 4000#сколько пар эмбеддинг-токен проходит через tail adapter за один forward
    elif architecture == 'I':
        #сколько субмоделей выбрасывается из forward на обучении. Чем больше, тем лучше защита от оверфита. Если одна-две субмодели, net_dropout лучше занулять
        net_dropout_rate = 0.0#если десятки субмоделей и хотим огромную защиту от оверфита, то это число надо проставлять в 0.9-0.95
        #дропаут внутри субмоделей
        individ_dropout_rate = 0.05
        conservativity = 0.15#насколько сильно подавляем отклонение от старой стратегии
        accum_batch = 30#60#для дачала сделайте 2
        lr = 1e-5
        
        layer_configs = [2048] * 6 + [1024] * 6#версия I
        use_memnets = True
        memnet_params={'num_heads':8, 'query_size':64, 'num_key_values':300, 'value_size':256}

        window_sizes = [8] * 8
        squeeze_dim = embedding_size//2#выходной размер слайдера
        
        composition_size = 1#число субмоделей класса ResNet. Чем больше их, тем больше или защита от оверфита (если большой net_dropout), или ёмкость
        sample_features = 1.#какая доля фичей приходит в субмодель. По дефолту ставится 0.4-0.6, но это если субмоделей как минимум 15. Сейчас их 1.
        path2model = "ern_model_E_composed_slider.pth"
        max_tokens_in_loss = 90#сколько пар эмбеддинг-токен проходит через tail adapter за один forward
    else:
        1/0

    #я подобрал перебором это значение batch_size и accum_batch, именно при нём быстрее всего проходим батч размера 2000. Я не знаю, почему, но это так.
    batch_size = 1#2

    
    if learnable_all:
        #опасный и тормозной режим
        batch_size = 2
        layer_configs = [512, 256]

path2slidermodel = path2model + '.slider'

In [9]:
device

device(type='cuda')

In [10]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, cache_dir="D:\cache\huggingface\\"+ model_name)

if bits_per_number == 4:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4"#, bnb_4bit_compute_dtype=torch.bfloat16
    )
elif bits_per_number == 8:
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True, bnb_8bit_use_double_quant=True, bnb_8bit_quant_type="nf8"#, bnb_8bit_compute_dtype=torch.bfloat16
    )
else:
    bnb_config = None

tokenizer.pad_token = tokenizer.eos_token
if learnable_all and not start_train:
    model = AutoModelForCausalLM.from_pretrained(
                  './last_modell',
                  #device_map="auto",
                  #device_map=device,
                  #torch_dtype=torch.bfloat16,
                  quantization_config=bnb_config,
                  cache_dir="D:\cache\huggingface\\"+ model_name)
else:
    model = AutoModelForCausalLM.from_pretrained(
                  model_name,
                  #device_map="auto",
                  #device_map=device,
                  #torch_dtype=torch.bfloat16,
                  quantization_config=bnb_config,
                  cache_dir="D:\cache\huggingface\\"+ model_name)
if use_gguf:
    model = PeftModelForCausalLM.from_pretrained(model, gguf_folder_name, load_in_4bit =True)
    if isinstance(model, PeftModelForCausalLM):
        model = model.merge_and_unload()


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
C:\Users\sd\AppData\Roaming\Python\Python39\site-packages\transformers\quantizers\auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [11]:
#model.lm_head
torch.save(model.lm_head.weight, "lin_model.pth")

In [12]:
class InstructDataset(Dataset):
    def __init__(self, data_file, tokenizer, max_seq_length, cut=None):
        self.data_file = data_file
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length
        self.cut = cut
        self.data = self.load_data()
        self.log_samples = deque(maxlen=45)

    def load_data(self):
        with open(self.data_file, 'rb') as f:
            data = pickle.load(f)
        random.seed(seed)
        data = random.sample(data, len(data))
        # if self.cut is None:
        #     data = [data_cur for data_cur in data if len(data_cur[1])>5]
        # else:
        #     data = [data_cur for data_cur in data[:self.cut] if self.cut>5]
        # for i in range(100):
        #     print('***', data[i])
        #     try:
        #         if (not 'Алис' in data[i][0]) and (not 'Элеон"' in data[i][0]) and (not 'Софи"' in data[i][0]) and (not 'Сэм"' in data[i][0]) (not 'Алис' in data[i][1]) and (not 'Элеон"' in data[i][1]) and (not 'Софи"' in data[i][1]) and (not 'Сэм"' in data[i][1]):
        #             print('***', data[i])
        #     except Exception:
        #         pass
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        self.log_samples.append(self.data[idx])
        parts = self.data[idx]
        text = parts[0]
        label = parts[1]
        if text is None:
            text = label
        
        # Кодируем текст и метку с помощью tokenizer
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']
        
        # Кодируем метку
        label_encoding = self.tokenizer.encode_plus(
            label,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        label_ids = label_encoding['input_ids']
        return {
            'input_ids': input_ids[0],
            'attention_mask': attention_mask,
            'labels': label_ids[0]
        }

In [13]:
class InstructDatasetR(Dataset):
    def __init__(self, data_file, tokenizer, max_seq_length, cut=None):
        self.data_file = data_file
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length
        self.cut = cut
        self.data = self.load_data()
        self.log_samples = deque(maxlen=45)

    def load_data(self):
        with open(self.data_file, 'rb') as f:
            data = pickle.load(f)
        random.seed(seed)
        data = random.sample(data, len(data))
        # if self.cut is None:
        #     data = [data_cur for data_cur in data if len(data_cur[1])>5]
        # else:
        #     data = [data_cur for data_cur in data[:self.cut] if self.cut>5]
        # for i in range(100):
        #     print('***', data[i])
        #     try:
        #         if (not 'Алис' in data[i][0]) and (not 'Элеон"' in data[i][0]) and (not 'Софи"' in data[i][0]) and (not 'Сэм"' in data[i][0]) (not 'Алис' in data[i][1]) and (not 'Элеон"' in data[i][1]) and (not 'Софи"' in data[i][1]) and (not 'Сэм"' in data[i][1]):
        #             print('***', data[i])
        #     except Exception:
        #         pass
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        self.log_samples.append(self.data[idx])
        parts = self.data[idx]
        text = parts[0]
        label = parts[1]
        parts = [parts[0], parts[1], 1]
        for r_variant in [-2,-1,-0.5,0.5,1,2]:
            s = f"<r{r_variant}>"
            if s in label:
                parts[-1] = r_variant
                label = label.replace(s, '')
                break
        r = parts[-1]
        if text is None:
            text = label
        
        # Кодируем текст и метку с помощью tokenizer
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']
        
        # Кодируем метку
        label_encoding = self.tokenizer.encode_plus(
            label,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        label_ids = label_encoding['input_ids']
        return {
            'input_ids': input_ids[0],
            'attention_mask': attention_mask,
            'labels': label_ids[0],
            'mult': torch.tensor(r)
        }

In [14]:
# Создание Dataset
if mode == 'pretrain':
    dataset = InstructDatasetR("../llms_local/dataset_llm_full_mflat.pkl", tokenizer, max_seq_len, cut=None)
else:
    dataset = InstructDatasetR("../llms_local/dataset_llm_full_instruct.pkl", tokenizer, max_seq_len, cut=None)
len(dataset)

420956

In [19]:
if start_train:
    #создать модель
    torch.manual_seed(1)
    np.random.seed(1)
    random.seed(1)
    slider = slider_utils.StackedSlidingWindow(input_dim=embedding_size, 
                                  output_dim=embedding_size, 
                                  window_sizes=window_sizes, 
                                  dropout_prob=individ_dropout_rate,
                                  squeeze_dim=squeeze_dim)
    
    head = ensembles.EResNetPro(input_size=embedding_size, 
               out_size=cardinality, 
               net_dropout_rate=net_dropout_rate, 
               individ_dropout_rate=individ_dropout_rate,
               layer_configs=layer_configs, 
               use_sigmoid_end=False, 
               use_batchnorm=True, 
               use_activation=True, 
               activation=nn.LeakyReLU(), 
               sample_features=sample_features, 
               composition_size=composition_size, 
               lin_bottleneck_size=None,
               lin_model_add=nn.Linear(embedding_size, cardinality).to(device),
               use_memnets=use_memnets,
               memnet_params=memnet_params)
    head.submodels[-1].weight = torch.nn.Parameter(torch.load( "lin_model.pth").to(device).to(torch.float32))
    head.submodels[-1].weight.requires_grad = learnable_linear_model
else:
    head = torch.load(path2model, weights_only=False)
    slider = torch.load(path2slidermodel, weights_only=False)

#собрать
if learnable_all:
    model.train()
    for param in model.parameters():
        try:
            param.requires_grad = True
            print('scss')
        except Exception as e:
            print(e)
else:
    model.eval()
head.train()
slider.train()
head.to(device)
slider.to(device)
head.by_submodels = True
optimizer = torch.optim.Adam(list(head.parameters()) + list(slider.parameters()), lr=lr)
#momentum = 0.9
#optimizer = torch.optim.SGD(list(head.parameters()) + list(slider.parameters()), lr=lr, momentum=momentum)

SlidingWindowLinear(
  (linear): Linear(in_features=12288, out_features=1024, bias=True)
)
SlidingWindowLinear(
  (linear): Linear(in_features=12288, out_features=1024, bias=True)
)
SlidingWindowLinear(
  (linear): Linear(in_features=12288, out_features=1024, bias=True)
)
SlidingWindowLinear(
  (linear): Linear(in_features=12288, out_features=1024, bias=True)
)
SlidingWindowLinear(
  (linear): Linear(in_features=12288, out_features=1024, bias=True)
)
use_memnets True


In [ ]:
def soft_clip(tens, mn, mx):
    mn = torch.tensor(mn).to(tens.device)
    mx = torch.tensor(mx).to(tens.device)
    tens[tens<mn] = torch.nn.Sigmoid()(tens[tens<mn]) + mn
    tens[tens>mx] = torch.nn.Sigmoid()(tens[tens>mx]) + mx
    return tens

In [ ]:
loss_fct = nn.CrossEntropyLoss(reduction = 'none')

In [ ]:
# Функция для подсчёта количества последних токенов, равных padding_token
def count_padding(tensor, padding_token):
    counts = []
    for row in tensor:
        count = 0
        for token in reversed(row):
            if token.item() == padding_token:
                count += 1
            else:
                break
        counts.append(count)
    return min(counts)

In [ ]:
proba_drop_idx = 0.02
proba_pad_idx = 0.1#некоторые токены мы замещаем паддингами. Модель должна всё равно работать. Кроме того, это надо для спекулятивного кодирования

In [ ]:
#проинференсить модель на батчах, сделать датасет для постпроцессинга
print('batch_size', batch_size)
i_pointer = 0

batch_accum_counter = 0
loss_array = []
metr_array = []
while 1: 
    input_ids_full = []
    labels_full = []
    weights_full = []
    #for i in range(batch_size):
    i = 0
    offset_counter = 0
    while i < batch_size:
        offset_counter += 1
        sample_cur  = dataset[i_pointer + i]
        input_ids = sample_cur['input_ids']
        labels = sample_cur['labels']
        weights = sample_cur['mult']
        
        input_ids_len = len(input_ids)
        labels_len = len(labels)
        input_ids = input_ids[input_ids!=padding_token]
        
        drop_idx = torch.rand_like(input_ids.to(torch.float)) < proba_drop_idx
        input_ids = input_ids[~drop_idx]
        
        labels = labels[labels!=padding_token]
        padding_size = input_ids_len + labels_len - len(input_ids) - len(labels)
        input_ids_cur = torch.cat([input_ids, labels, torch.zeros(padding_size, dtype=torch.int32) + padding_token])[:max_seq_len4inference]
        labels_cur = torch.cat([input_ids * 0 + padding_token, labels, torch.zeros(padding_size, dtype=torch.int32) + padding_token])[:max_seq_len4inference]
        #print('fact_size', np.min([len(input_ids) + len(labels), max_seq_len4inference]), len(input_ids) + len(labels))

        if torch.all(labels_cur) == padding_token:
            print('empty label')
            continue
        i += 1
        
        input_ids_full.append(input_ids_cur[:-1])
        labels_full.append(labels_cur[1:])
        weights_full.append(weights)

        # if not torch.any(input_ids!=0):
        #     #X + Y
        #     print(list(input_ids_cur.numpy()))
        #     print(list(labels_cur.numpy()))
        #     1/0
    i_pointer += i #should be batch_size
    #print('A', pd.Timestamp.now())
    with torch.no_grad():
        input_ids_full = torch.stack(input_ids_full).to('cuda')
        labels_full = torch.stack(labels_full).to('cuda')
        weights_full = torch.stack(weights_full).to('cuda').T

        cnt_pads = count_padding(labels_full, padding_token)
        if cnt_pads >= input_ids_full.shape[1] - 5:
            continue
        if cnt_pads > 0:
            input_ids_full = input_ids_full[:, :-cnt_pads]
            labels_full = labels_full[:, :-cnt_pads]

        #рандомные паддинги, для спекулятивки
        to_pad_idx = torch.rand_like(input_ids_full.to(torch.float)) < proba_pad_idx
        input_ids_full[to_pad_idx] = padding_token
        #
        #outp = model.forward(input_ids_full, attention_mask=None, labels=labels_full, token_type_ids=None, write_caches=False, read_caches=False, return_states=True)
        # head.by_submodels = False
        # head.training = False
        # model.lm_head = head
        # model.lm_head.half()
        # outp = model.forward(input_ids_full, output_hidden_states=True, return_dict=False)

        outp = model.forward(input_ids_full, output_hidden_states=True, return_dict=True)
        outp['states'] = outp['hidden_states'][-1]
    # {"loss": loss_agg, "logits": logits, "states":embeddings}
        outp['states'] = outp['states'][:, :input_ids_full.shape[1]]
        
        
        #state_cur = outp['states'].reshape([outp['states'].shape[0] * outp['states'].shape[1], outp['states'].shape[2]])#.cpu().to(torch.float16).numpy()
        weights_full2d = torch.vstack([weights_full] * labels_full.shape[-1]).to('cuda').T
        labels_cur = labels_full.ravel()#.cpu().numpy()
        weights_cur = weights_full2d.ravel()
        state_cur_raw = outp['states']
        
        del labels_full
        del weights_full2d
        del weights_full
        del outp
        idx = torch.isin(labels_cur, torch.tensor(forbidden_tokens_list).to(device))
        labels_cur = labels_cur[~idx]


    

    state_cur_slided = slider.forward(state_cur_raw.to(torch.float32))
    shape_cur_slided = state_cur_slided.shape
    state_cur = state_cur_slided.reshape((shape_cur_slided[0] * shape_cur_slided[1], shape_cur_slided[2]))
    del state_cur_slided
    del state_cur_raw
    state_cur = state_cur[~idx]
    weights_cur = weights_cur[~idx]
        # if len(labels_cur) > max_tokens_in_loss:
        #     indices = np.random.choice(len(labels_cur), size=max_tokens_in_loss, replace=False)
        #     labels_cur = labels_cur[indices]
        #     state_cur = state_cur[indices]
    #print('B', pd.Timestamp.now())
    if state_cur.shape[0] == 0:
        continue
    for x_start_pointer in range(0, state_cur.shape[0], max_tokens_in_loss): 
        logits, lst_logits = head(state_cur.to(torch.float32)[x_start_pointer: x_start_pointer + max_tokens_in_loss])
        
        ce_loss = loss_fct(logits, labels_cur[x_start_pointer: x_start_pointer + max_tokens_in_loss].view(-1))
        ce_loss = ce_loss * weights_cur[x_start_pointer: x_start_pointer + max_tokens_in_loss].to('cuda')
        ce_loss = soft_clip(ce_loss, -0.6, 10.)
        loss_ampl_head = torch.mean(torch.abs(lst_logits[-1] - logits)) / (torch.std(lst_logits[-1]) + 1)
        
        loss = torch.mean(ce_loss) * (1./(conservativity + 1.)) + loss_ampl_head * (conservativity/(conservativity + 1.))
        loss.backward(retain_graph=True)
        l = len(lst_logits)
        for j in range(l - 1, -1, -1):
            del lst_logits[j]
        #print('-C', pd.Timestamp.now())
        #del lst_logits
    del state_cur
    #print('D', pd.Timestamp.now())

    acc = torch.mean((torch.argmax(logits, axis=-1) == labels_cur[x_start_pointer: x_start_pointer + max_tokens_in_loss]).to(torch.float16))
    del logits
    del labels_cur
    batch_accum_counter += 1
    loss_array += [torch.mean(ce_loss).item()]
    metr_array += [acc.item()]
    if batch_accum_counter >= accum_batch:
        optimizer.step()
        optimizer.zero_grad()
        torch.cuda.empty_cache()
        print('ce_loss', np.round(np.mean(loss_array), 3), 'loss_ampl_head', np.round(loss_ampl_head.item(), 4), i_pointer, 'from', len(dataset), np.round(100 * i_pointer/len(dataset), 5), '%', 'acc', np.mean(metr_array), pd.Timestamp.now())
        loss_array = []
        metr_array = []
        batch_accum_counter = 0

        
        
        if (np.random.rand() < 1 and accum_batch > 100) or (np.random.rand() < 0.1 and accum_batch <= 100):
            head.training = False
            torch.save(head, path2model)
            torch.save(slider, path2slidermodel)
            if np.random.rand()<0.2:
                torch.save(head, path2model + '.back')
                torch.save(slider, path2slidermodel + '.back')
            head.training = True
            if learnable_all:
                model.eval()
                model.save_pretrained("./last_modell")
                model.train()
                for param in model.parameters():
                    try:
                        param.requires_grad = True
                    except Exception as e:
                        pass
            

In [ ]:
state_cur.shape, idx.shape

In [ ]:
#если принудительно стопнули, можно добить батч
optimizer.step()
optimizer.zero_grad()
torch.cuda.empty_cache()
print('ce_loss', np.round(np.mean(loss_array), 3), 'loss_ampl_head', np.round(loss_ampl_head.item(), 4), i_pointer, 'from', len(dataset), np.round(100 * i_pointer/len(dataset), 5), '%', 'acc', np.mean(metr_array), pd.Timestamp.now())

In [ ]:
#сохранение tail adapter-а вручную
head.training = False
head.by_submodels = False
torch.save(head, path2model)
torch.save(slider, path2slidermodel)

if learnable_all:
    model.eval()
    model.save_pretrained("./last_modell")

In [ ]:
head

In [20]:
slider.half()
head.half()
head.by_submodels = False

In [21]:
prompt = "Привет, как дела?"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
# Generate
temp = 0.9
top_p=0.01
max_new_tokens = 150
repetition_penalty = 1.2
top_k = 10

t = pd.Timestamp.now()
generate_ids = generate_utils.generate_speculative(model, inputs.input_ids.to(device), slider=slider, heavy_lm_head=head,
             top_p=top_p, temperature=temp, max_new_tokens=max_new_tokens, 
             pad_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id, bos_token_id=tokenizer.bos_token_id, 
             do_sample=True, repetition_penalty=repetition_penalty, early_stopping=False, 
             tokenizer=tokenizer, stop_strings=None, top_k=top_k, 
             return_dict_in_generate=False, use_cache=True, estimation_rule='0.2')
print(pd.Timestamp.now() - t)
answer = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
answer

0 days 00:00:13.539772


'?art�\'moursер�Iingи at\n# d�xОode\n\n\u200b�String..(""тyie notNg��t;\r\nv //be\r\nsl n�7iпange�\r\n\r\n\r\nt�im ву�с();\nе&#Vq\\вA();\nо g1� н$ o7нes���$�млcccanpunctionvalto �is� P, iipsn а\x15�##itceuress seemsim�riics uos//ap s;kikanthaceox]ool# e�+ t\rressac\x0eߠ ,�ur3mSrascth hhe'

In [22]:
prompt = "Привет, как дела?"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
# Generate
temp = 0.9
top_p=0.01
max_new_tokens = 150
repetition_penalty = 1.2
top_k = 10

t = pd.Timestamp.now()
generate_ids = generate_utils.generate_speculative(model, inputs.input_ids.to(device), 
             slider=None, heavy_lm_head=None,
             top_p=top_p, temperature=temp, max_new_tokens=max_new_tokens, 
             pad_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id, bos_token_id=tokenizer.bos_token_id, 
             do_sample=True, repetition_penalty=repetition_penalty, early_stopping=False, 
             tokenizer=tokenizer, stop_strings=None, top_k=top_k, 
             return_dict_in_generate=False, use_cache=True, estimation_rule='0.2')
print(pd.Timestamp.now() - t)
answer = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
answer

0 days 00:00:08.590997


"?I, kidding responding bit you response am'm just Just been busy things do life to. I for had time good day today also's in much better as yesterday was is evenWhat than the thing have like that what it not so nice but thanks very kind me your again andSorry you previous sorry no long one know about this thank already.\n\n late new be of still after here ago days more since when some are always we hope make now if done with't at happyven out all best i everything can help because well will try every single any get ask soon would need a maybe talk else other or my want share has us before never another end could did how please back yes tell then hi you hi hi friend dear hi hi hi hi hello\n\n hi"

In [ ]:
prompt = "Привет, как дела?"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
# Generate

t = pd.Timestamp.now()
generate_ids = model.generate(inputs.input_ids.to(device), top_p=top_p, temperature=temp, max_new_tokens=max_new_tokens, 
             pad_token_id=None, eos_token_id=None, bos_token_id=None, 
             do_sample=True, repetition_penalty=repetition_penalty, early_stopping=False, 
             tokenizer=tokenizer, stop_strings=None, top_k=top_k, 
             return_dict_in_generate=False, output_scores=False, use_cache=True)
print(pd.Timestamp.now() - t)
answer = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
answer